In [1]:
import sys; sys.path.insert(0,'..')
import pyzx as zx
from pyzx.generate import CNOT_HAD_PHASE_circuit

We often want to extract a circuit from a ZX-diagram. However, equivalent ZX-diagrams can induce drastically different circuits upon extraction even if they have similar numbers of spiders and edges. We can apply two optimization techniques, simulated annealing and genetic algorithms, to search this local space for a ZX-diagram that is more amenable to extraction.

Given a circuit, the typical approach to searching this space is to first fully simplify the associated ZX-diagram and then search over ZX-diagrams that are equivalent to this fully simplified version. So, let's first generate a random circuit and convert it to a ZX-diagram:

In [2]:
N_QUBITS = 5
DEPTH = 100
c = CNOT_HAD_PHASE_circuit(qubits=N_QUBITS, depth=DEPTH, clifford=False)
print(c.stats())
g = c.to_graph()

Circuit  on 5 qubits with 100 gates.
        20 is the T-count
        80 Cliffords among which 
        61 2-qubit gates (61 CNOT, 0 other) and
        19 Hadamard gates.


Then, we want to fully simplify this ZX-diagram. There are two ways of doing this: applying `full_reduce` or applying `teleport_reduce` followed by circuit-level optimizations via `basic_optimization`. Note that if you go with the latter, you have to put the resulting ZX-diagram in graph-like form with `zx.simplify.to_graph_like()`. We'll go with the former:

In [3]:
g_simp = g.copy()
zx.full_reduce(g_simp)

Now, we can search the space of equivalent ZX-diagrams. First lets do it with annealing:

In [31]:
N_ITERS = 1000

g_anneal, _ = zx.anneal(g_simp, iters=N_ITERS, full_reduce_prob=1.0)

annealing...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 113.85it/s]


Let's convert this new ZX-diagram back to a circuit and see how we did. We can apply some optimizations along the way:

In [32]:
zx.full_reduce(g_anneal)
c_anneal = zx.extract_circuit(g_anneal.copy()).to_basic_gates()
c_anneal = zx.basic_optimization(c_anneal)
print(c_anneal.stats())

Circuit  on 5 qubits with 69 gates.
        13 is the T-count
        56 Cliffords among which
        39 2-qubit gates (35 CNOT, 4 other) and
        14 Hadamard gates.


We can do the same with genetic algorithms

In [33]:
ga_opt = zx.GeneticOptimizer()
g_evolve = ga_opt.evolve(g_simp, n_mutants=20, n_generations=40, quiet=False)
zx.full_reduce(g_evolve)
c_evolve = zx.extract_circuit(g_evolve.copy()).to_basic_gates()
c_evolve = zx.basic_optimization(c_evolve)
print(c_evolve.stats())

Generations: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.26it/s]

Circuit  on 5 qubits with 69 gates.
        13 is the T-count
        56 Cliffords among which
        38 2-qubit gates (34 CNOT, 4 other) and
        14 Hadamard gates.


Typically, this approach works well for small circuits (i.e., < 10 qubits) but it depends on the structure of the circuit. Also, annealing tends to do better than the genetic approach.